In [58]:
from scraper.datamodel import DbApi
import pandas as pd
from collections import Counter
from contextlib import closing
import time
import numpy as np
import copy

In [3]:
db = DbApi('../data/backup2.db')

In [4]:
db.conn.execute('select count(*) from comments').fetchall()

[(1459660,)]

In [6]:
db.conn.execute('select count(*) from submissions').fetchall()

[(222211,)]

In [7]:
db.conn.execute('select count(*) from sentences').fetchall()

[(5816712,)]

In [15]:
db.conn.execute('select src_is_subm, count(*) from sentences group by 1').fetchall()

[(0, 2908670), (1, 2908042)]

In [22]:
db.conn.execute('select lower(text), count(*) from sentences where src_is_subm=1 group by 1 order by 2 desc limit 50').fetchall()

[("i don't know what to do.", 3465),
 ("i don't know.", 1931),
 ('i want to die.', 1780),
 ('i hate myself.', 1146),
 ('i need help.', 1099),
 ("i don't know what to do anymore.", 1090),
 ('i just want to die.', 1088),
 ('what do i do?', 977),
 ('please help me.', 872),
 ('what should i do?', 697),
 ('thanks for reading.', 678),
 ('i have no friends.', 675),
 ("i can't do this anymore.", 670),
 ("i'm so tired.", 607),
 ("what's the point?", 546),
 ("i just don't know what to do.", 524),
 ('i need help', 488),
 ("i can't take it anymore.", 428),
 ('i give up.', 428),
 ('i want to kill myself.', 424),
 ('what can i do?', 421),
 ("i don't know what to do", 405),
 ('i hate my life.', 399),
 ('i hate it.', 396),
 ("i don't want to die.", 395),
 ('i want to die', 363),
 ('i have nothing.', 340),
 ("i just don't know what to do anymore.", 332),
 ('please help me', 331),
 ("i can't do it anymore.", 328),
 ("i don't know why.", 316),
 ("i don't want to live anymore.", 315),
 ('i feel so alone.'

In [23]:
qry_nobots = """
with bots as (
    select author, count(*)
    from comments 
    group by 1
    having count(*) >=20
        union all
    select 'SeriouslyLucifer', -1
        union all
    select 'peadditer', -1
)
select lower(text), count(*) 
from sentences s
join comments c
  on s.src_id = c.id
  and src_is_subm=0 
left join bots b
  on c.author = b.author
where b.author is null
and c.author <> '[deleted]'
group by 1 order by 2 desc limit 50
"""
db.conn.execute(qry_nobots).fetchall()

[('thank you so much!', 355),
 ('username checks out', 210),
 ('i love it.', 205),
 ("i don't know.", 178),
 ('what do you mean?', 174),
 ('i like it.', 165),
 ('thank you :)', 163),
 ('thanks so much!', 142),
 ('happy cake day!', 132),
 ('i love it!', 125),
 ('not at all.', 121),
 ('thank you so much.', 121),
 ('thanks for sharing.', 116),
 ('thank you very much!', 113),
 ('best of luck!', 112),
 ('what are you talking about?', 111),
 ('thanks for the info.', 106),
 ("i don't get it.", 104),
 ("i don't think so.", 103),
 ('thanks for the reply.', 103),
 ('thanks a lot!', 98),
 ('thanks for the info!', 98),
 ('i agree with you.', 95),
 ('thanks for sharing!', 93),
 ('username checks out.', 92),
 ('that makes sense.', 90),
 ('( ͡° ͜ʖ ͡°)', 89),
 ('hope this helps!', 89),
 ('good to know.', 84),
 ('thank you for your submission!', 84),
 ('hope this helps.', 82),
 ('keep it up!', 81),
 ('thanks for the advice.', 79),
 ('enter pls bot', 76),
 ('i get it.', 75),
 ('yes it is.', 73),
 ('what

In [17]:
# These fucking guys.
qry_lowfreq_bots = """
select author, subreddit, s.text, count(*), max(c.body)
from comments c
join sentences s
  on  s.src_id = c.id
  and s.src_is_subm = 0
where s.text = ? or s.text = ? 
order by 1, 3, 4 desc, 2
"""
bad_sents = ['___\n\n[**Here are our subreddit rules.', "**Unfortunately, we've had to remove your post."]

db.conn.execute(qry_lowfreq_bots, bad_sents).fetchall()

[('SeriouslyLucifer',
  'FortniteCompetitive',
  "**Unfortunately, we've had to remove your post.",
  190,
  "**Unfortunately, we've had to remove your post.** \n\n___\n\n###Posts Should Be High Quality And Encourage Discussion\n\nWe want /r/FortniteCompetitive to be the best source of discussion for the game online - and to do that we have to ensure that quality content isn’t drowned out by low effort or low-quality submissions.  Content must be related to Fortnite. If your post contains a suggestion for the game, it must contain Fortnite elements.\n\n**High Quality Guidelines:**\n\n- Posts about similar games are not allowed.\n\n- Every post should be a space for meaningful discussion. While this doesn't mean every post needs to be a discursive essay, posts that don't offer room for meaningful discussion will be removed.\n\n- Screenshots are removed at moderation discretion if they do not contain sufficient supporting text to encourage a discussion.\n\n- Videos/images taken with a ph

In [19]:
# These fucking guys.
qry_lowfreq_bots = """
select author, subreddit, s.text, count(*), max(c.body)
from comments c
join sentences s
  on  s.src_id = c.id
  and s.src_is_subm = 0
where s.text = 'Thank you for your submission!'
order by 1, 3, 4 desc, 2
"""

db.conn.execute(qry_lowfreq_bots).fetchall()

[('peadditer',
  'CoDCompetitive',
  'Thank you for your submission!',
  87,
  "Thank you for your submission! Unfortunately, your submission has been removed for the following reason(s):\n\nPlease use the 'Daily Discussion Thread' stickied at the top of the subreddit for your question/topic. Thanks!\n\n\n\nIf you have any questions regarding the ruleset of /r/CoDCompetitive, please refer to the [FAQ](/r/CoDCompetitive/wiki/index) or [message the moderators](/message/compose?to=%2Fr%2FCoDCompetitive). Thank you!")]

In [40]:
start = time.time()

cnt_comm_tf = Counter()
cnt_comm_df = Counter()
with closing(db.conn.cursor()) as c:
    gen = c.execute('select body from comments')
    for i, (text,) in enumerate(gen):
        d = Counter()
        d.update(text.lower().split())
        cnt_comm_tf.update(d)
        cnt_comm_df.update(d.keys())
n_comm = i
print(time.time() - start)
print(i, len(cnt_comm_tf))

37.022057056427
1459659 1231462


In [41]:
start = time.time()

cnt_subm_tf = Counter()
cnt_subm_df = Counter()
with closing(db.conn.cursor()) as c:
    gen = c.execute('select title from submissions union all select selftext from submissions where is_self=1')
    for i, (text,) in enumerate(gen):
        d = Counter()
        d.update(text.lower().split())
        cnt_subm_tf.update(d)
        cnt_subm_df.update(d.keys())
n_subm = i
print(time.time() - start)
print(i, len(cnt_subm_tf))

25.065020322799683
442071 389304


In [36]:
cnt_comm_tf.most_common(100)

[('the', 1535466),
 ('to', 1032931),
 ('a', 965910),
 ('and', 859736),
 ('i', 852094),
 ('of', 644979),
 ('you', 564206),
 ('is', 537537),
 ('in', 489951),
 ('that', 488138),
 ('it', 461248),
 ('for', 392180),
 ('but', 287617),
 ('on', 276145),
 ('with', 265918),
 ('have', 261598),
 ('be', 259680),
 ('this', 255160),
 ('not', 243111),
 ('are', 237965),
 ('was', 234033),
 ('if', 225946),
 ('my', 214425),
 ('they', 212317),
 ('as', 206037),
 ('just', 201495),
 ('so', 186079),
 ('like', 185412),
 ('your', 177553),
 ('or', 176903),
 ('at', 159017),
 ('can', 146429),
 ('he', 144158),
 ('what', 135837),
 ("it's", 132615),
 ('do', 127583),
 ('would', 126773),
 ('get', 125770),
 ('all', 125128),
 ('about', 124421),
 ('from', 123035),
 ('an', 116234),
 ('one', 110885),
 ('more', 109193),
 ('when', 106976),
 ('we', 106957),
 ('people', 106579),
 ('me', 106100),
 ('will', 103837),
 ('out', 98474),
 ("don't", 97211),
 ('no', 95474),
 ('up', 95251),
 ('think', 94488),
 ('because', 93774),
 ('how', 

In [35]:
cnt_comm_df.most_common(10)

[('the', 607960),
 ('a', 500870),
 ('to', 484192),
 ('i', 446699),
 ('and', 429680),
 ('of', 355243),
 ('is', 343057),
 ('you', 316606),
 ('in', 307512),
 ('that', 304376)]

In [38]:
cnt_subm_tf.most_common(100)

[('i', 2729366),
 ('to', 1555544),
 ('and', 1295577),
 ('the', 978861),
 ('my', 928848),
 ('a', 892719),
 ('of', 655955),
 ('that', 485079),
 ('in', 465111),
 ('me', 443298),
 ('it', 430352),
 ("i'm", 424492),
 ('but', 413693),
 ('have', 403322),
 ('for', 400996),
 ('is', 373703),
 ('just', 357488),
 ('was', 319448),
 ('with', 300913),
 ('this', 294883),
 ('so', 283325),
 ("don't", 267547),
 ('be', 265008),
 ('not', 235051),
 ('like', 225542),
 ('on', 216634),
 ('want', 216514),
 ('do', 192767),
 ('about', 192056),
 ('feel', 187287),
 ('all', 186784),
 ('at', 185382),
 ('know', 184379),
 ('or', 178878),
 ('if', 172133),
 ("i've", 171944),
 ('no', 168679),
 ('she', 168357),
 ('because', 161338),
 ('what', 158311),
 ('been', 158135),
 ('get', 154434),
 ('you', 153517),
 ('am', 153327),
 ('as', 153125),
 ('out', 146065),
 ('even', 137461),
 ('had', 135263),
 ('they', 135083),
 ('up', 133053),
 ('myself', 130957),
 ('would', 130030),
 ("can't", 129783),
 ('he', 129008),
 ('when', 127983),


In [49]:
subm_tfidf = pd.Series(cnt_subm_tf) * np.log(n_subm / pd.Series(cnt_subm_df))

In [50]:
comm_tfidf = pd.Series(cnt_comm_tf) * np.log(n_comm / pd.Series(cnt_comm_df))

In [47]:
comm_tfidf.sort_values(ascending=False)[:100]

i          4.756062e+06
to         2.880121e+06
and        2.698952e+06
the        2.081177e+06
my         1.971378e+06
a          1.896979e+06
of         1.469842e+06
that       1.203252e+06
in         1.109475e+06
me         1.064368e+06
i'm        1.012210e+06
it         1.009906e+06
but        9.904810e+05
have       9.753725e+05
for        9.652693e+05
was        9.101507e+05
is         8.807353e+05
just       8.443554e+05
with       7.764649e+05
this       7.351770e+05
so         7.269982e+05
don't      6.885982e+05
be         6.829914e+05
not        6.216999e+05
she        6.192748e+05
like       6.061851e+05
on         5.846428e+05
want       5.657265e+05
about      5.334879e+05
at         5.287717e+05
               ...     
only       3.174303e+05
time       3.031340e+05
we         3.010988e+05
who        2.935400e+05
go         2.917812e+05
an         2.891671e+05
has        2.876623e+05
now        2.834155e+05
more       2.800192e+05
it.        2.755803e+05
being      2.663

In [52]:
ratio_test = (subm_tfidf / comm_tfidf).sort_values(ascending=False)

ratio_test[:100]

[deleted]             182.727158
suicidewatch          120.278985
pm]                   110.119156
r/suicidewatch         92.942063
celexa                 73.914837
self-harming           70.294110
/r/suicidewatch.       68.988035
suicidal...            68.408304
worthlessness.         67.388547
stranger:              66.233942
hotline?               65.412474
harming,               65.341704
suicidal.              63.246380
loner,                 61.877547
depressed/suicidal     58.166769
suicidal?              57.901848
ged,                   57.787929
hotline,               57.323919
painlessly.            57.192177
suicide..              57.120605
r/depression           56.082649
feeling:               55.260702
**edit:                54.492214
ideation               51.476674
depressive,            51.312489
hospitalization,       49.783331
offing                 49.641127
everynight             49.176217
/r/depression          49.107322
self-hatred.           48.569103
          

In [54]:
ratio_test[ratio_test.notnull()][-100:]

rgb          0.005441
thicc        0.005366
mech         0.005323
🙄            0.005317
pvp          0.005316
😅            0.005307
leftists     0.005300
emperor      0.005297
vai          0.005278
ja           0.005234
av           0.005108
alliance     0.005092
devs         0.005076
faction      0.005056
melee        0.005056
ports        0.005014
za           0.004987
rework       0.004977
democrats    0.004951
você         0.004932
elo          0.004837
nu           0.004813
hier         0.004805
nato         0.004754
paladin      0.004745
ele          0.004704
factions     0.004653
warranty     0.004648
alle         0.004598
8gb          0.004582
               ...   
f2p          0.003228
gtx          0.003110
så           0.003082
dann         0.003080
&gt;if       0.003063
alexa        0.003047
raids        0.002969
sie          0.002918
bei          0.002913
&gt;the      0.002739
wie          0.002718
för          0.002652
goku         0.002641
mai          0.002542
che       

In [57]:
vocab = set(cnt_subm_tf.keys())
vocab.update(cnt_comm_tf.keys())
print(len(cnt_subm_tf), len(cnt_comm_tf), len(vocab))
print(len(vocab)-len(cnt_subm_tf), len(vocab)-len(cnt_comm_tf))

389304 1231462 1437628
1048324 206166


In [64]:
# Super duper naive bayes!
subm_tf = copy.deepcopy(cnt_subm_tf)
subm_df = copy.deepcopy(cnt_subm_df)
comm_tf = copy.deepcopy(cnt_comm_tf)
comm_df = copy.deepcopy(cnt_comm_df)

subm_tf.update(vocab)
subm_df.update(vocab)
comm_tf.update(vocab)
comm_df.update(vocab)

subm_tf = pd.Series(subm_tf)
subm_df = pd.Series(subm_df)
comm_tf = pd.Series(comm_tf)
comm_df = pd.Series(comm_df)

In [96]:
subm_df_logprob = np.log(subm_df) - np.log(subm_df.sum())
subm_tf_logprob = np.log(subm_tf) - np.log(subm_tf.sum())
comm_df_logprob = np.log(comm_df) - np.log(comm_df.sum())
comm_tf_logprob = np.log(comm_tf) - np.log(comm_tf.sum())

In [98]:
def query(text, loglike, logprior=np.log(.5), V=None):
    if V is None:
        V = Counter(loglike.index)
    X =  Counter(text.lower().split())
    X.update(V) # if V is series, use .to_dict()
    X = pd.Series(X)
    return logprior + loglike.dot(X)
        

In [99]:
query("i want to kill myself", subm_tf_logprob, np.log(n_subm/(n_comm+n_subm))), query("i want to kill myself", comm_tf_logprob, np.log(n_comm/(n_comm+n_subm))) 

(-24848622.913981337, -23878461.093272228)

In [100]:
query("i want to kill myself", subm_tf_logprob), query("i want to kill myself", comm_tf_logprob) 

(-24848622.148079742, -23878461.521858256)

In [102]:
query("i want to kill myself", subm_df_logprob), query("i want to kill myself", comm_df_logprob) 

(-24075007.102122888, -23598643.151869409)

In [101]:
subm_tf_logprob.sum(), comm_tf_logprob.sum()

(-24848597.404012207, -23878429.597757116)

In [103]:
subm_df_logprob.sum(), comm_df_logprob.sum()

(-24074979.335168492, -23598610.521870237)